## Authentication

- Creating CogniteClient using different methods
  - Interactive login
  - Using Device code
  - Using clientID &amp; client secret
- Checking the login status

You can authenticate the Python SDK with Azure AD by using a token retrieved when a user authenticates or with a static client secret for long-running jobs like extractors or calculations.

### Prerequisites

* Install the Microsoft Authentication Library (MSAL) for Python.

In [ ]:
#! pip install msal
#! poetry add msal

* You need to specify the values for the following configuration parameters:
 * `Tenant ID` - the ID of the Azure AD tenant where the user is registered.
 * `Client ID` - the ID of the application in Azure AD.
 * `Cluster` - the cluster where your CDF project is installed. For example, api and westeurope-1.
 * `CDF project` - the name of the CDF project.

If you don't know which values to use for these variables, contact your CDF administrator or Cognite Support.

You can directly set the values for these parameters here or read as environment variables or from file.

In [ ]:
TENANT_ID="48d5043c-cf70-4c49-881c-c638f5796997"
CLIENT_ID="fab52bb5-9de2-4f9e-aefa-712da4b5fe00"
CDF_CLUSTER="westeurope-1"
COGNITE_PROJECT="ds-basics"

Also set the following derived variables, which will be used for obtaining token

In [ ]:
SCOPES = [f"https://{CDF_CLUSTER}.cognitedata.com/.default"]

AUTHORITY_HOST_URI = "https://login.microsoftonline.com"
AUTHORITY_URI = AUTHORITY_HOST_URI + "/" + TENANT_ID
PORT = 53000

TOKEN_URL = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"

### Authenticate with user credentials
You can authenticate the Python SDK with Azure AD by using a token retrieved with user credentials.

#### Interactive Login
You can get the token by letting the user sign in interactively via a browser and use the authenticate with interactive login and token refresh flow to access CDF when you're running short-term scripts or using Jupyter.

In [ ]:
from cognite.client import CogniteClient
from msal import PublicClientApplication

In [ ]:
app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)
creds = app.acquire_token_interactive(scopes=SCOPES, port=PORT)

In [ ]:
client = CogniteClient(
    token_url=creds["id_token_claims"]["iss"],
    token=creds["access_token"],
    token_client_id=creds["id_token_claims"]["aud"],
    project=COGNITE_PROJECT,
    base_url=f"https://{CDF_CLUSTER}.cognitedata.com",
    client_name="cognite-client-interactive"
    )

In [ ]:
#client.iam.token.inspect()

#### Using Device Code
If a browser is not available, for example, if you are logged into a terminal, you can use the authenticate with user credentials and device code flow.

In [ ]:
app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)
device_flow = app.initiate_device_flow(scopes=SCOPES)
print(device_flow["message"])  # print device code to screen
creds = app.acquire_token_by_device_flow(flow=device_flow)

In [ ]:
client = CogniteClient(
    token_url=creds["id_token_claims"]["iss"],
    token=creds["access_token"],
    token_client_id=creds["id_token_claims"]["aud"],
    project=COGNITE_PROJECT,
    base_url=f"https://{CDF_CLUSTER}.cognitedata.com",
    client_name="cognite-client-device",
)

### Authenticate with client secret

Make Sure, You've created an environment variable CLIENT_SECRET with the value of the client secret obtained from admin.

In [ ]:
from dotenv import load_dotenv
from pathlib import Path

In [ ]:
# Obtain the Environment Variables from .env file
dotenv_path = Path(".env")
load_dotenv(dotenv_path=dotenv_path)

In [ ]:
import os
CLIENT_SECRET = os.getenv("CLIENT_SECRET")  # store secret in env variable

Or you can also use getpass to enter the value in notebook interactively. (Not recommended)

In [ ]:
#from getpass import getpass
#CLIENT_SECRET = getpass("Enter the Client Secret: ")  # Enter the client secret interactively here

In [ ]:
client = CogniteClient(
    client_name="client_secret_test",
    token_url=TOKEN_URL,
    token_client_id=CLIENT_ID,
    token_client_secret=CLIENT_SECRET,
    token_scopes=SCOPES,
    project=COGNITE_PROJECT,
    base_url=f"https://{CDF_CLUSTER}.cognitedata.com",
    debug=False,
)

### Check the login status

In [ ]:
client.login.status()

### Saving the Authentication Code in Script for Later Usage

Let's define the function for each authentication method.

In [ ]:
def interactive_client():
    """Function to Create the Cognite Client, using Interactive Login method
    """
    app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)
    creds = app.acquire_token_interactive(scopes=SCOPES, port=PORT)
    client = CogniteClient(
        token_url=creds["id_token_claims"]["iss"],
        token=creds["access_token"],
        token_client_id=creds["id_token_claims"]["aud"],
        project=COGNITE_PROJECT,
        base_url=f"https://{CDF_CLUSTER}.cognitedata.com",
        client_name="cognite-client-interactive"
        )
    return client

In [ ]:
def device_code_client():
    """Function to Create the Cognite Client, using Device code method
    """
    app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)
    device_flow = app.initiate_device_flow(scopes=SCOPES)
    print(device_flow["message"])  # print device code to screen
    creds = app.acquire_token_by_device_flow(flow=device_flow)
    client = CogniteClient(
        token_url=creds["id_token_claims"]["iss"],
        token=creds["access_token"],
        token_client_id=creds["id_token_claims"]["aud"],
        project=COGNITE_PROJECT,
        base_url=f"https://{CDF_CLUSTER}.cognitedata.com",
        client_name="cognite-client-device",
        )
    return client

In [ ]:
def client_secret_client():
    """Function to Create the Cognite Client, using Credentials (e.g. ClientID, Client secret)
    """
    client = CogniteClient(
        token_url=TOKEN_URL,
        token_client_id=CLIENT_ID,
        token_client_secret=CLIENT_SECRET,
        token_scopes=SCOPES,
        project=COGNITE_PROJECT,
        base_url=f"https://{CDF_CLUSTER}.cognitedata.com",
        client_name="client_secret_test_script",
        debug=False,
        )
    return client

Finally create a wrapper function to create the cognite client, which takes method as the argument.

In [ ]:
def create_cognite_client(method="interactive-login") -> CogniteClient:
    """Function to Create the Client

    Args:
        method (str, optional): One of the methods ("interactive-login","device-code","client-secret"). Defaults to "interactive-login".

    Returns:
        CogniteClient: CogniteClient to be used to access Cognite Data Fusion.
    """
    if method=="interactive-login":
        client = interactive_client()
    elif method=="device-code":
        client = device_code_client()
    elif method=="client-secret":
        client = client_secret_client()
    else:
        client = None
        print("Client couldn't be created. Specify one of these methods :interactive-login, device-code, client-secret")
    return client


Finally let's test these methods

In [ ]:
c = create_cognite_client(method="interactive-login")

In [ ]:
c = create_cognite_client(method="device-code")

In [ ]:
c = create_cognite_client(method="client-secret")

## Check Login Status

In [ ]:
c.login.status()